In [3]:
test_mp4, test_depth, test_intr = "./raw_data/rgb_20250711_040525", "./raw_data/depth_20250711_040525", "./raw_data/intrinsics_20250711_040525"

In [6]:
import cv2
from frame_processor import FrameProcessor
from mock_camera import MockCamera

from config import Config 

camera_intrinsics = Config.CAMERA_INTRINSICS
#Config.CAMERA_INTRINSICS

camera = MockCamera(rgb_folder=test_mp4, depth_folder=test_depth, intrinsics_folder=test_intr)
processor = FrameProcessor(camera_intrinsics)
processor.set_tracking_enabled(True)
camera.start()

# Обработка и отображение
while True:
   color, depth, depth_intrinsics= camera.get_frames()
   if color is None or depth is None:
       break
   
   # Обработка кадра
   results = processor.process_frame(color, depth, depth_intrinsics)
   
   # Отображение результата (combined = RGB + Depth + Cleaned)
   cv2.imshow('Processed Video', results['combined'])
   
   # Или только обработанный RGB с масками
   # cv2.imshow('Processed Video', results['color_with_mask'])
   
   if cv2.waitKey(25) & 0xFF == ord('q'):
       break

camera.stop()
cv2.destroyAllWindows()

# Вывод статистики
print(f"Обработано кадров: {processor.frame_count}")

TOP DIMENSIONS:
0: [     2.9191     0.92214     0.66489]
1: [     2.7979     0.88151     0.70807]
2: [     4.1678     0.94238     0.69636]
3: [     4.2319      1.0125     0.69727]
4: [     4.0001      1.0999     0.95944]
5: [     4.2163      1.0316     0.87749]
Object 0: габариты 2.919x0.922x0.665 м, объем 1.7897 м³, 27535 точек
TOP DIMENSIONS:
0: [    0.86726     0.53558     0.51799]
1: [     1.1897     0.62926     0.38229]
2: [     1.1129      0.5612     0.52835]
3: [     1.2374     0.52676     0.50636]
4: [     1.3394     0.59129     0.58519]
5: [     1.2542     0.54741     0.50976]
Object 1: габариты 0.867x0.536x0.518 м, объем 0.2406 м³, 24226 точек
[FILTER] Area change too large: 0.64
Обработано кадров: 71


In [7]:
import numpy as np
completed_objects = processor.get_completed_objects()
print(f"\nЗавершено объектов: {len(completed_objects)}")

# Анализ результатов
for idx, obj in enumerate(completed_objects):
    print(f"\n Объект №: {idx}")
    print(f"  Точек в облаке: {len(obj['point_cloud'])}")
    print(f"  Всего масок: {len(obj['all_masks'])}")
    print(f"  Габариты: {obj['dimensions'][0]:.3f}x{obj['dimensions'][1]:.3f}x{obj['dimensions'][2]:.3f} м, ")


Завершено объектов: 2

 Объект №: 0
  Точек в облаке: 27535
  Всего масок: 21
  Габариты: 2.919x0.922x0.665 м, 

 Объект №: 1
  Точек в облаке: 24226
  Всего масок: 35
  Габариты: 0.867x0.536x0.518 м, 


In [8]:
import open3d as o3d
import numpy as np

def visualize_point_cloud_with_bbox(result):
    """Визуализация облака точек с ориентированным bounding box"""
    # Создаем облако точек
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(result['point_cloud'])
    pcd.paint_uniform_color([0.5, 0.5, 0.5])
    
    # Создаем линии для отображения bounding box
    bbox_lines = create_bbox_lines(result['centroid'], result['dimensions'], result['rotation_matrix'])
    
    # Текст с размерами
    print(f"Габариты: {result['dimensions'][0]:.3f} x {result['dimensions'][1]:.3f} x {result['dimensions'][2]:.3f} м")
    print(f"Объем: {result['volume']:.4f} м³")
    print(f"Точек: {result['num_points']}")
    
    # Визуализация
    o3d.visualization.draw_geometries(
        [pcd, bbox_lines],
        window_name="Облако точек с габаритами",
        width=800, height=600
    )

def create_bbox_lines(center, extent, rotation):
    """Создание линий bounding box"""
    # Вершины box в локальных координатах
    corners = np.array([
        [-extent[0]/2, -extent[1]/2, -extent[2]/2],
        [extent[0]/2, -extent[1]/2, -extent[2]/2],
        [extent[0]/2, extent[1]/2, -extent[2]/2],
        [-extent[0]/2, extent[1]/2, -extent[2]/2],
        [-extent[0]/2, -extent[1]/2, extent[2]/2],
        [extent[0]/2, -extent[1]/2, extent[2]/2],
        [extent[0]/2, extent[1]/2, extent[2]/2],
        [-extent[0]/2, extent[1]/2, extent[2]/2]
    ])
    
    # Поворот и смещение
    corners = (rotation @ corners.T).T + center
    
    # Индексы линий
    lines = [[0,1],[1,2],[2,3],[3,0],
             [4,5],[5,6],[6,7],[7,4],
             [0,4],[1,5],[2,6],[3,7]]
    
    # Создание LineSet
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(corners)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in lines])
    
    return line_set

In [10]:
visualize_point_cloud_with_bbox(completed_objects[1])

Габариты: 0.867 x 0.536 x 0.518 м
Объем: 0.2406 м³
Точек: 24226


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(completed_objects[0]['rgb_image'])